# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
source_file = '../output_data/output_cities_AR.csv'
cities_df = pd.read_csv(source_file,sep = ",", low_memory=False)
cities_df['Date'] = pd.to_datetime(cities_df['Date'], unit='s')
cities_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Leżajsk,50.26,22.42,54.00,62,100,12.75,PL,2020-05-03 17:59:38
1,Salé,34.04,-6.82,96.80,19,20,5.82,MA,2020-05-03 17:56:51
2,Talnakh,69.49,88.40,23.00,92,100,17.90,RU,2020-05-03 17:59:47
3,Khandyga,62.67,135.60,20.01,94,89,4.92,RU,2020-05-03 17:59:47
4,Busselton,-33.65,115.33,66.99,76,48,12.57,AU,2020-05-03 17:59:47
...,...,...,...,...,...,...,...,...,...
560,Talcahuano,-36.72,-73.12,60.80,77,75,1.12,CL,2020-05-03 18:02:20
561,Dawlatabad,36.41,64.91,75.72,34,17,11.18,AF,2020-05-03 18:02:20
562,Port Shepstone,-30.74,30.45,66.11,79,5,9.93,ZA,2020-05-03 17:58:31
563,Mufumbwe,-13.68,24.80,72.61,39,81,3.27,ZM,2020-05-03 18:02:21


In [3]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Data columns (total 9 columns):
City          565 non-null object
Latitude      565 non-null float64
Longitude     565 non-null float64
Max Temp      565 non-null float64
Humidity      565 non-null int64
Cloudiness    565 non-null int64
Wind Speed    565 non-null float64
Country       561 non-null object
Date          565 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 39.9+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df['Humidity']

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,
                                 weights = humidity, dissipating = False,
                                 max_intensity = 300, point_radius = 5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
hotel_df = cities_df.loc[(cities_df['Max Temp'] < 80) & 
                         (cities_df['Max Temp'] > 70) 
                              & (cities_df['Wind Speed'] < 10) & (cities_df['Humidity'] <=50)].copy() # I like clouds

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# * Store into variable named `hotel_df`.
# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ""

hotel_df.reset_index(drop=True).head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Provincia di Imperia,43.97,7.78,73.40,42,40,9.17,IT,2020-05-03 18:00:32,
1,Sawākin,19.11,37.33,77.94,48,0,1.50,SD,2020-05-03 18:00:34,
2,Muros,42.78,-9.06,78.01,50,83,1.99,ES,2020-05-03 18:00:37,
3,Mopipi,-21.18,24.88,73.36,21,0,5.66,BW,2020-05-03 18:00:52,
4,Kununurra,-15.77,128.73,71.60,28,90,6.93,AU,2020-05-03 18:01:05,


In [8]:
# TARGET / COORDINATES
radius = 5000

# URL
base_url_nearby = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# PARAMETERS
params = {
    "types": "lodging",
    "radius": radius,
    "key": g_key}

for index,row in hotel_df.iterrows():
    lat=row["Latitude"]
    lng=row["Longitude"]
    params["location"]=f"{lat},{lng}"
    
# RUN REQUEST
    response = requests.get(base_url_nearby, params)
    
# # TEST RESULTS
#     print(response.url)
    results = response.json()
#    print(results["results"][0]["name"])
# print(results["results"][0]["vicinity"])

# # PRINT RESULTS

    try:
     # record the hotel name
        hotel_df['Hotel Name'][index] = results["results"][0]["name"]
    except IndexError:
        print(f"No hotels available near {hotel_df['City'][index]}.\n")
        hotel_df['Hotel Name'][index] = "none available"
hotel_df

C:\Users\Andrea Rose\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


No hotels available near Mopipi.



C:\Users\Andrea Rose\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


No hotels available near Lumeje.

No hotels available near Mufumbwe.



,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
144,Provincia di Imperia,43.97,7.78,73.40,42,40,9.17,IT,2020-05-03 18:00:32,La Stregatta
153,Sawākin,19.11,37.33,77.94,48,0,1.50,SD,2020-05-03 18:00:34,فندق الدويم
162,Muros,42.78,-9.06,78.01,50,83,1.99,ES,2020-05-03 18:00:37,Apartment Av. Castelao
218,Mopipi,-21.18,24.88,73.36,21,0,5.66,BW,2020-05-03 18:00:52,none available
268,Kununurra,-15.77,128.73,71.60,28,90,6.93,AU,2020-05-03 18:01:05,Hotel Kununurra
280,Sabha,27.04,14.43,78.93,14,0,0.65,LY,2020-05-03 18:01:08,محطة ام الارانب -سبها مواصلات
314,La Esperanza,14.87,-91.57,75.00,40,11,4.00,GT,2020-05-03 18:01:18,Hotel Imperio Inn
336,Akdepe,42.06,59.38,78.80,27,90,4.70,TM,2020-05-03 18:01:24,Мой дом
365,Torrington,41.80,-73.12,73.99,45,44,4.00,US,2020-05-03 17:57:04,The Yankee Pedlar Inn
375,Douglas,41.30,-96.17,72.00,33,90,4.70,US,2020-05-03 17:58:09,Hilton Garden Inn Omaha West


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>La Stregatta</dd>\n<dt>City</dt><dd>Provincia di Imperia</dd>\n<dt>Country</dt><dd>IT</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>فندق الدويم</dd>\n<dt>City</dt><dd>Sawākin</dd>\n<dt>Country</dt><dd>SD</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Apartment Av. Castelao</dd>\n<dt>City</dt><dd>Muros</dd>\n<dt>Country</dt><dd>ES</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>none available</dd>\n<dt>City</dt><dd>Mopipi</dd>\n<dt>Country</dt><dd>BW</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Kununurra</dd>\n<dt>City</dt><dd>Kununurra</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>محطة ام الارانب -سبها مواصلات</dd>\n<dt>City</dt><dd>Sabha</dd>\n<dt>Country</dt><dd>LY</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Imperio Inn</dd>\n<dt>City</dt><dd>La Esperanza</dd>\n<dt>Country</dt><dd>GT</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Мой дом</dd>\n<dt>City</dt><dd>Akdepe</dd>\n<dt>Country</dt><dd>TM</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>

In [10]:
new_map = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(new_map)
fig

Figure(layout=FigureLayout(height='420px'))